In [19]:
import numpy as np
import pandas as pd
import os, time
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [20]:
dataset_dir = 'C://project//dataset/kwix/curation'
user_data = pd.read_csv(dataset_dir + '/user.csv')
user_data = user_data.to_numpy()
user_data[:,1] = (user_data[:,1]-100)/100
user_data[:,2] = (user_data[:,2]-30)/60
user_data[:,4] = (user_data[:,4])/60
user_data[:,5] = (user_data[:,5]-10)/30
user_data

array([[1.00000000e+00, 8.00000000e-01, 5.83333333e-01, ...,
        1.66666667e-01, 3.35390947e-01, 4.00000000e+00],
       [2.00000000e+00, 7.00000000e-01, 5.83333333e-01, ...,
        5.00000000e-01, 4.16378316e-01, 2.00000000e+00],
       [3.00000000e+00, 5.50000000e-01, 2.50000000e-01, ...,
        3.33333333e-01, 2.91016302e-01, 3.00000000e+00],
       ...,
       [4.98000000e+02, 6.00000000e-01, 5.83333333e-01, ...,
        8.33333333e-01, 5.13020833e-01, 1.00000000e+00],
       [4.99000000e+02, 5.70000000e-01, 6.33333333e-01, ...,
        8.33333333e-01, 5.86244202e-01, 1.00000000e+00],
       [5.00000000e+02, 8.10000000e-01, 8.00000000e-01, ...,
        6.66666667e-01, 4.60293235e-01, 2.00000000e+00]])

In [21]:
chest_df = pd.read_csv(dataset_dir +'/chest.csv')
chest = pd.read_csv(dataset_dir +'/chest.csv').to_numpy()[:, 1:]
lower = pd.read_csv(dataset_dir + '/lower_body.csv').to_numpy()[:, 1:]
abdom = pd.read_csv(dataset_dir + '/abdominals.csv').to_numpy()[:, 1:]
chest

array([[0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 1, ..., 0, 0, 1],
       [1, 0, 1, ..., 0, 0, 1],
       [1, 0, 1, ..., 0, 0, 1]], dtype=int64)

In [22]:
x_train, x_val, y_train, y_val = train_test_split(user_data[:,1:7], chest, test_size=0.1, random_state=2022)
x_train

array([[0.78      , 0.86666667, 1.        , 0.33333333, 0.52935235,
        3.        ],
       [0.69      , 0.61666667, 1.        , 0.33333333, 0.44861875,
        3.        ],
       [0.78      , 0.68333333, 0.        , 0.83333333, 0.41362623,
        1.        ],
       ...,
       [0.83      , 0.4       , 1.        , 0.5       , 0.20415659,
        3.        ],
       [0.65      , 0.43333333, 0.        , 0.66666667, 0.35231099,
        2.        ],
       [0.42      , 0.68333333, 1.        , 0.5       , 0.84037559,
        2.        ]])

In [23]:
model = Sequential([
    Dense(16, activation='tanh', input_shape = (6,)),
    Dense(32, activation='tanh'),
    Dense(15, activation='tanh')
])

In [58]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['mae'])
model.summary()

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[
        ModelCheckpoint('models/curation_model.h5', monitor='mae', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='mae', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                112       
                                                                 
 dense_7 (Dense)             (None, 32)                544       
                                                                 
 dense_8 (Dense)             (None, 15)                495       
                                                                 
Total params: 1,151
Trainable params: 1,151
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
 1/15 [=>............................] - ETA: 7s - loss: 0.4534 - mae: 0.6455
Epoch 00001: mae improved from inf to 0.62288, saving model to models\curation_model.h5
15/15 [==============================] - 1s 16ms/step - loss: 0.3756 - mae: 0.6229 - val_loss: 0.3667 - val_mae: 0.6193 - lr: 0.0010
Epoch

In [59]:
height = 180
weight = 85
sex = 1
age = 20    
bmi = weight*10000 /height**2 
ability = 4
height = (height-100)/100
weight = (weight-30)/60
age = (age)/60
bmi = (bmi-10)/30

In [60]:
y_pred = model.predict([[ height, weight, sex, age, bmi, ability]])
print(y_pred)
np.argmax(y_pred)
y_bull = y_pred >= 0.7
rec_list = []
print(y_bull)
for i in range(0,15):
    if y_bull[0,i]:
        rec_list.append(chest_df.columns[i+1])
        
rec_list


[[-0.7522003  -0.99996686 -0.3175812  -0.85684067 -0.9886015  -0.95164007
  -0.42147684  0.0930692   0.09059088 -0.35824004 -0.6424242  -0.63274425
   0.81362253 -0.6621597   1.        ]]
[[False False False False False False False False False False False False
   True False  True]]


['플랫푸쉬업', '벤트오버덤벨레이즈']